In [1]:
#Reading appropriate libraries 

import tensorflow as tf
from tensorflow import keras

#Library for binarized neural network
import larq as lq
import pandas as pd
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

2022-11-27 00:58:14.285142: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-27 00:58:15.042053: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/saumya/.conda/envs/tf/lib/
2022-11-27 00:58:15.042161: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/saumya/.conda/envs/tf/lib/
2022-11-27 00:58:15.042169: W tensorflow/compiler/tf2tensorrt/

In [2]:
#Reading banknote authentication dataset for this code

df = pd.read_csv('/home/saumya/Downloads/banknote_authentication.csv', sep = ';')

In [3]:
#Extracting the needed information
xdata = df[['wavelet_transformed_variance', 'wavelet_transformed_skewness', 'wavelet_transformed_curtosis','image_entropy']].to_numpy()
ydata = df[['counterfeit']].to_numpy()

In [4]:
#Here the error from the approximate processors are simulated in the dataset.
#Here we are simulating a random error of +,- 10% 
error_mat = np.ones(np.shape(xdata))
for i in range(error_mat.shape[0]):
    for j in range(error_mat.shape[1]):
        error_mat[i][j] = xdata[i][j]*0.1*rand.random()*(-1)**(rand.randint(0, 1))
        
#new data with added error on it        
xdata_error = xdata + error_mat

In [5]:
#Fitting the data 
scaler = StandardScaler()
xtrain = scaler.fit_transform(xdata)
xtrain_error = scaler.fit_transform(xdata_error)

In [6]:
#Splitting the data for trainning and testing
xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size = 0.2, train_size=0.8)
xtrain_error, xtest_error, ytrain_error, ytest_error = train_test_split(xdata_error, ydata, test_size = 0.2, train_size=0.8)

In [7]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

#Defining model with no added error
model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
#Input Layer
model.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Second Layer
model.add(lq.layers.QuantDense(8, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Third Layer
model.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
#Output layer
model.add(lq.layers.QuantDense(1,activation='sigmoid', use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))

2022-11-27 00:58:16.448877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 00:58:16.458365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 00:58:16.458690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 00:58:16.459329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [8]:
#Defining the model to train with error added data

model_error = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
#Input Layer
model_error.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Second Layer
model_error.add(lq.layers.QuantDense(8, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Third Layer
model_error.add(lq.layers.QuantDense(4, use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))
#Output layer
model_error.add(lq.layers.QuantDense(1,activation='sigmoid', use_bias=False, **kwargs))
model_error.add(tf.keras.layers.BatchNormalization(scale=False))

In [9]:
#Compiling the model with no added error

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(xtrain, ytrain, batch_size=64, epochs=1000)

test_loss, test_acc = model.evaluate(xtest, ytest)

Epoch 1/1000
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2022-11-27 00:58:19.073467: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x1da014e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-11-27 00:58:19.073513: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2022-11-27 00:58:19.078023: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-11-27 00:58:19.141283: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-27 00:58:19.175318: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 [==============================] - 3s 5ms/step - loss: 8.5673 - accuracy: 0.2361
Epoch 2/1000
18/18 [==============================] - 0s 4ms/step - loss: 9.1579 - accuracy: 0.2479
Epoch 3/1000
18/18 [==============================] - 0s 4ms/step - loss: 7.2558 - accuracy: 0.2106
Epoch 4/1000
18/18 [==============================] - 0s 5ms/step - loss: 8.3526 - accuracy: 0.2334
Epoch 5/1000
18/18 [==============================] - 0s 4ms/step - loss: 8.1383 - accuracy: 0.2261
Epoch 6/1000
18/18 [==============================] - 0s 5ms/step - loss: 8.3702 - accuracy: 0.2407
Epoch 7/1000
18/18 [==============================] - 0s 4ms/step - loss: 8.2100 - accuracy: 0.2443
Epoch 8/1000
18/18 [==============================] - 0s 4ms/step - loss: 7.9767 - accuracy: 0.2461
Epoch 9/1000
18/18 [==============================] - 0s 4ms/step - loss: 8.1594 - accuracy: 0.2343
Epoch 10/1000
18/18 [==============================] - 0s 4ms/step - loss: 8.1322 - accuracy: 0.2370
Epoch 11/100

18/18 [==============================] - 0s 4ms/step - loss: 4.3452 - accuracy: 0.6171
Epoch 83/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.3414 - accuracy: 0.6235
Epoch 84/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.3282 - accuracy: 0.6363
Epoch 85/1000
18/18 [==============================] - 0s 4ms/step - loss: 5.0823 - accuracy: 0.6108
Epoch 86/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.4398 - accuracy: 0.6208
Epoch 87/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.6556 - accuracy: 0.6326
Epoch 88/1000
18/18 [==============================] - 0s 5ms/step - loss: 4.6730 - accuracy: 0.6217
Epoch 89/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.2820 - accuracy: 0.5971
Epoch 90/1000
18/18 [==============================] - 0s 4ms/step - loss: 5.1360 - accuracy: 0.6226
Epoch 91/1000
18/18 [==============================] - 0s 4ms/step - loss: 5.1539 - accuracy: 0.6217
Epoc

18/18 [==============================] - 0s 5ms/step - loss: 3.8156 - accuracy: 0.6718
Epoch 163/1000
18/18 [==============================] - 0s 5ms/step - loss: 4.1702 - accuracy: 0.6390
Epoch 164/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.8892 - accuracy: 0.6655
Epoch 165/1000
18/18 [==============================] - 0s 5ms/step - loss: 4.0686 - accuracy: 0.6481
Epoch 166/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.6066 - accuracy: 0.6655
Epoch 167/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.8506 - accuracy: 0.6481
Epoch 168/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.6902 - accuracy: 0.6582
Epoch 169/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.8122 - accuracy: 0.6527
Epoch 170/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.6761 - accuracy: 0.6609
Epoch 171/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.8341 - accuracy: 0.

...

18/18 [==============================] - 0s 4ms/step - loss: 1.8129 - accuracy: 0.7776
Epoch 643/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.4788 - accuracy: 0.7949
Epoch 644/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.1834 - accuracy: 0.7758
Epoch 645/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.5359 - accuracy: 0.8077
Epoch 646/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.4699 - accuracy: 0.7840
Epoch 647/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.4826 - accuracy: 0.7912
Epoch 648/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6071 - accuracy: 0.7037
Epoch 649/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6265 - accuracy: 0.7220
Epoch 650/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7627 - accuracy: 0.7384
Epoch 651/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6056 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 1.3699 - accuracy: 0.6746
Epoch 723/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.3196 - accuracy: 0.6673
Epoch 724/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.5941 - accuracy: 0.7211
Epoch 725/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.8363 - accuracy: 0.7293
Epoch 726/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.3258 - accuracy: 0.6964
Epoch 727/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.2198 - accuracy: 0.6691
Epoch 728/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.5407 - accuracy: 0.6928
Epoch 729/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.2222 - accuracy: 0.6910
Epoch 730/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.3101 - accuracy: 0.7192
Epoch 731/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.2188 - accuracy: 0.

18/18 [==============================] - 0s 3ms/step - loss: 1.4571 - accuracy: 0.7849
Epoch 803/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.8227 - accuracy: 0.7785
Epoch 804/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.5849 - accuracy: 0.7821
Epoch 805/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7821 - accuracy: 0.7639
Epoch 806/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7636 - accuracy: 0.7812
Epoch 807/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6946 - accuracy: 0.7748
Epoch 808/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9613 - accuracy: 0.7703
Epoch 809/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7688 - accuracy: 0.7739
Epoch 810/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7425 - accuracy: 0.7776
Epoch 811/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.8094 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 1.7644 - accuracy: 0.7794
Epoch 883/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.7577 - accuracy: 0.7758
Epoch 884/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7766 - accuracy: 0.7767
Epoch 885/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7111 - accuracy: 0.7803
Epoch 886/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7888 - accuracy: 0.7794
Epoch 887/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.7786 - accuracy: 0.7794
Epoch 888/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7700 - accuracy: 0.7776
Epoch 889/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.7555 - accuracy: 0.7867
Epoch 890/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7538 - accuracy: 0.7840
Epoch 891/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7689 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 1.8348 - accuracy: 0.7739
Epoch 963/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7403 - accuracy: 0.7840
Epoch 964/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6704 - accuracy: 0.7603
Epoch 965/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7513 - accuracy: 0.7721
Epoch 966/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6453 - accuracy: 0.7730
Epoch 967/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.3448 - accuracy: 0.7521
Epoch 968/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6216 - accuracy: 0.7603
Epoch 969/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7874 - accuracy: 0.7758
Epoch 970/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.0219 - accuracy: 0.7721
Epoch 971/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7592 - accuracy: 0.

In [10]:
#Compiling the model with added error

model_error.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_error.fit(xtrain_error, ytrain_error, batch_size=64, epochs=1000)

test_loss_error, test_acc_error = model.evaluate(xtest_error, ytest_error)

Epoch 1/1000
18/18 [==============================] - 2s 4ms/step - loss: 6.3971 - accuracy: 0.4467
Epoch 2/1000
18/18 [==============================] - 0s 4ms/step - loss: 6.3347 - accuracy: 0.4686
Epoch 3/1000
18/18 [==============================] - 0s 4ms/step - loss: 6.7582 - accuracy: 0.4859
Epoch 4/1000
18/18 [==============================] - 0s 4ms/step - loss: 6.4037 - accuracy: 0.4868
Epoch 5/1000
18/18 [==============================] - 0s 4ms/step - loss: 6.7789 - accuracy: 0.5242
Epoch 6/1000
18/18 [==============================] - 0s 4ms/step - loss: 5.7698 - accuracy: 0.6007
Epoch 7/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.8072 - accuracy: 0.6199
Epoch 8/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.4523 - accuracy: 0.5834
Epoch 9/1000
18/18 [==============================] - 0s 4ms/step - loss: 5.0672 - accuracy: 0.6016
Epoch 10/1000
18/18 [==============================] - 0s 4ms/step - loss: 5.9445 - accuracy: 0.5716

18/18 [==============================] - 0s 4ms/step - loss: 5.1978 - accuracy: 0.3947
Epoch 83/1000
18/18 [==============================] - 0s 5ms/step - loss: 5.7980 - accuracy: 0.4120
Epoch 84/1000
18/18 [==============================] - 0s 5ms/step - loss: 5.7318 - accuracy: 0.4184
Epoch 85/1000
18/18 [==============================] - 0s 5ms/step - loss: 5.0944 - accuracy: 0.4148
Epoch 86/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.8910 - accuracy: 0.4275
Epoch 87/1000
18/18 [==============================] - 0s 5ms/step - loss: 5.8805 - accuracy: 0.3829
Epoch 88/1000
18/18 [==============================] - 0s 5ms/step - loss: 4.5168 - accuracy: 0.4485
Epoch 89/1000
18/18 [==============================] - 0s 4ms/step - loss: 5.2598 - accuracy: 0.4057
Epoch 90/1000
18/18 [==============================] - 0s 5ms/step - loss: 5.2923 - accuracy: 0.4284
Epoch 91/1000
18/18 [==============================] - 0s 5ms/step - loss: 6.0088 - accuracy: 0.4202
Epoc

18/18 [==============================] - 0s 5ms/step - loss: 2.9499 - accuracy: 0.5916
Epoch 163/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.6647 - accuracy: 0.6080
Epoch 164/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.5285 - accuracy: 0.6235
Epoch 165/1000
18/18 [==============================] - 0s 5ms/step - loss: 4.8592 - accuracy: 0.5889
Epoch 166/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.1709 - accuracy: 0.6135
Epoch 167/1000
18/18 [==============================] - 0s 5ms/step - loss: 4.2763 - accuracy: 0.5798
Epoch 168/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.3887 - accuracy: 0.6026
Epoch 169/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.3721 - accuracy: 0.6080
Epoch 170/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.3176 - accuracy: 0.6235
Epoch 171/1000
18/18 [==============================] - 0s 5ms/step - loss: 3.4652 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 3.4478 - accuracy: 0.6290
Epoch 243/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.6981 - accuracy: 0.5934
Epoch 244/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.4005 - accuracy: 0.6080
Epoch 245/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.5173 - accuracy: 0.6126
Epoch 246/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.9593 - accuracy: 0.6199
Epoch 247/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.3638 - accuracy: 0.6317
Epoch 248/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.3232 - accuracy: 0.6335
Epoch 249/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.1697 - accuracy: 0.6272
Epoch 250/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.6618 - accuracy: 0.5789
Epoch 251/1000
18/18 [==============================] - 0s 4ms/step - loss: 7.0425 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 2.9815 - accuracy: 0.6490
Epoch 323/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.7614 - accuracy: 0.6472
Epoch 324/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.3554 - accuracy: 0.6199
Epoch 325/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.0915 - accuracy: 0.6153
Epoch 326/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.0552 - accuracy: 0.6272
Epoch 327/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.1322 - accuracy: 0.6180
Epoch 328/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.6464 - accuracy: 0.6235
Epoch 329/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.3898 - accuracy: 0.6217
Epoch 330/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.0926 - accuracy: 0.6144
Epoch 331/1000
18/18 [==============================] - 0s 4ms/step - loss: 4.2642 - accuracy: 0.

...

18/18 [==============================] - 0s 4ms/step - loss: 2.5702 - accuracy: 0.6855
Epoch 643/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.4600 - accuracy: 0.7010
Epoch 644/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.3342 - accuracy: 0.7220
Epoch 645/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.1180 - accuracy: 0.7466
Epoch 646/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.7059 - accuracy: 0.7147
Epoch 647/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7536 - accuracy: 0.6591
Epoch 648/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.8919 - accuracy: 0.7010
Epoch 649/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.6182 - accuracy: 0.7284
Epoch 650/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.2105 - accuracy: 0.7220
Epoch 651/1000
18/18 [==============================] - 0s 3ms/step - loss: 3.3811 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 3.5175 - accuracy: 0.7575
Epoch 723/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.6990 - accuracy: 0.7502
Epoch 724/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.4502 - accuracy: 0.7347
Epoch 725/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.3792 - accuracy: 0.7511
Epoch 726/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.4062 - accuracy: 0.7493
Epoch 727/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.4611 - accuracy: 0.7420
Epoch 728/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.6269 - accuracy: 0.7366
Epoch 729/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.6524 - accuracy: 0.7420
Epoch 730/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.5774 - accuracy: 0.7530
Epoch 731/1000
18/18 [==============================] - 0s 4ms/step - loss: 3.7370 - accuracy: 0.

18/18 [==============================] - 0s 5ms/step - loss: 1.6194 - accuracy: 0.7776
Epoch 803/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9046 - accuracy: 0.7685
Epoch 804/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6581 - accuracy: 0.7612
Epoch 805/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.2940 - accuracy: 0.7466
Epoch 806/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6351 - accuracy: 0.7995
Epoch 807/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9492 - accuracy: 0.7603
Epoch 808/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9904 - accuracy: 0.7940
Epoch 809/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9390 - accuracy: 0.7438
Epoch 810/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6826 - accuracy: 0.7521
Epoch 811/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.2854 - accuracy: 0.

18/18 [==============================] - 0s 3ms/step - loss: 1.5067 - accuracy: 0.7639
Epoch 883/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.6614 - accuracy: 0.8049
Epoch 884/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.8206 - accuracy: 0.7903
Epoch 885/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.5445 - accuracy: 0.7812
Epoch 886/1000
18/18 [==============================] - 0s 3ms/step - loss: 1.6386 - accuracy: 0.7767
Epoch 887/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.4722 - accuracy: 0.8140
Epoch 888/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7961 - accuracy: 0.7840
Epoch 889/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.5371 - accuracy: 0.7894
Epoch 890/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.8268 - accuracy: 0.7748
Epoch 891/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9053 - accuracy: 0.

18/18 [==============================] - 0s 4ms/step - loss: 3.1569 - accuracy: 0.7247
Epoch 963/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.8285 - accuracy: 0.7338
Epoch 964/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7201 - accuracy: 0.7521
Epoch 965/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.6737 - accuracy: 0.7411
Epoch 966/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7280 - accuracy: 0.7758
Epoch 967/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.8043 - accuracy: 0.7758
Epoch 968/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.7435 - accuracy: 0.7612
Epoch 969/1000
18/18 [==============================] - 0s 4ms/step - loss: 2.0238 - accuracy: 0.7393
Epoch 970/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9602 - accuracy: 0.7511
Epoch 971/1000
18/18 [==============================] - 0s 4ms/step - loss: 1.9391 - accuracy: 0.

In [11]:
lq.models.summary(model)

+sequential stats--------------------------------------------------------------------+
| Layer                  Input prec.  Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs |
|                              (bit)               x 1       x 1    (kB)             |
+------------------------------------------------------------------------------------+
| quant_dense                      1  (-1, 4)       16         0    0.00          16 |
| batch_normalization              -  (-1, 4)        0         8    0.03           0 |
| quant_dense_1                    1  (-1, 8)       32         0    0.00          32 |
| batch_normalization_1            -  (-1, 8)        0        16    0.06           0 |
| quant_dense_2                    1  (-1, 4)       32         0    0.00          32 |
| batch_normalization_2            -  (-1, 4)        0         8    0.03           0 |
| quant_dense_3                    1  (-1, 1)        4         0    0.00           4 |
| batch_normalization_3            -  (-1, 

In [12]:
#The output is still accuarate with any added variation in data.
print(f"The accuracy with no added random error is: {test_acc * 100:.2f} %")

The accuracy with no added random error is: 80.36 %


In [13]:
print(f"The accuracy with added random error is:  {test_acc_error * 100:.2f} %")

The accuracy with added random error is:  79.27 %
